<a href="https://colab.research.google.com/github/suchievement/d2l-pytorch/blob/master/zsod_2018150418.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation + Import


In [2]:
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-yssnh32y
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-yssnh32y
     |████████████████████████████████| 53 kB 587 kB/s 
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369408 sha256=89af17fe5d30d211849cdf3817f5841e355f7f34061a02917a0888c1fd7c2488
  Stored in directory: /tmp/pip-ephem-wheel-cache-ai1y20be/wheels/ab/4f/3a/5e51521b55997aa6f0690e095c08824219753128ce8d9969a3
Successfully built clip


In [3]:
!pip install selectivesearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for selectivesearch: filename=selectivesearch-0.4-py3-none-any.whl size=4350 sha256=26a6ccbd4c20fb2c39cfb8d22bb0e521e50563196c8736caa7a9e7f8d7b14f9e
  Stored in directory: /root/.cache/pip/wheels/66/4e/88/6de23ce74be839a953498c4ebdfa809ad7da9422ac89ae856c
Successfully built selectivesearch


In [4]:
!pip3 install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/philferriere/cocoapi.git to /tmp/pip-install-wasa6exq/pycocotools_132df2b7c1f744e8b156f2644c0f9124
  Running command git clone -q https://github.com/philferriere/cocoapi.git /tmp/pip-install-wasa6exq/pycocotools_132df2b7c1f744e8b156f2644c0f9124
  Created wheel for pycocotools: filename=pycocotools-2.0-cp38-cp38-linux_x86_64.whl size=304781 sha256=e9158b3e580ea5177f005d78171ff1c18e5107d4bfe77892cbb54b522795b727
  Stored in directory: /tmp/pip-ephem-wheel-cache-741_bk6o/wheels/bd/1c/0d/8c82e1b9bc855b82e1eb53eadea4459efe171d2daf5a222701
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.6
    Uninstalling pycocotools-2.0.6:
      Successfully uninstalled pycocotools-2.0.6


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import clip
from selectivesearch import selective_search
import cv2
from PIL import Image
#import fiftyone as fo
from pycocotools.coco import COCO
import os.path
from typing import Any, Callable, Optional, Tuple, List
import json
from PIL import Image

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [7]:
%cd /content/drive/MyDrive/22_2_DL/data

[Errno 2] No such file or directory: '/content/drive/MyDrive/22_2_DL/data'
/content


In [42]:
# clip
model, preprocess = clip.load("ViT-L/14", device=device)

100%|███████████████████████████████████████| 890M/890M [00:18<00:00, 51.7MiB/s]


# COCO Dataset 준비

In [28]:
root_dir = "/content/drive/MyDrive/22_2_DL/data/"
data_path = "/content/drive/MyDrive/22_2_DL/data/val2017/"
labels_path = "/content/drive/MyDrive/22_2_DL/data/annotations/instances_val2017.json"

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
coco = COCO(labels_path)

loading annotations into memory...
Done (t=0.73s)
creating index...
index created!


In [31]:
coco.loadImgs(285)

[{'license': 4,
  'file_name': '000000000285.jpg',
  'coco_url': 'http://images.cocodataset.org/val2017/000000000285.jpg',
  'height': 640,
  'width': 586,
  'date_captured': '2013-11-18 13:09:47',
  'flickr_url': 'http://farm8.staticflickr.com/7434/9138147604_c6225224b8_z.jpg',
  'id': 285}]

# 0 CLIP의 텍스트 인코더로 각 class에 대한 임베딩 생성


In [43]:
# pycocotools를 이용해 names를 리스트에 저장
names=[]
cats = coco.loadCats(coco.getCatIds())
names.append([cat['name'] for cat in cats])
names = names[0]

In [44]:
# 80개 category에 대한 텍스트 임베딩을 구함
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in names]).to(device)
text_features = model.encode_text(text_inputs)
text_features = text_features.float()

In [45]:
# 80개의 768차원 벡터 생성
text_features.shape

torch.Size([80, 768])

#1 ss의 output 좌표를 coco형식으로 바꾸는 함수
- ss,COCO의 output은 (left, top, w, h)
- PIL은 (left, top, right, bottom)
  - ss[0],ss[1],ss[0]+ss[2],ss[1]+ss[3] 으로 바꾸기


In [117]:
# 1. ss에서 나온 좌표를 bboxes라는 리스트에 담아 return하는 함수
def stc(selectivesearch_output):
  bboxes = []
  for ss in selectivesearch_output:
    bboxes.append([ss['rect'][0],ss['rect'][1],ss['rect'][0]+ss['rect'][2],ss['rect'][1]+ss['rect'][3]])
  return bboxes

In [118]:
# 2. gt를 바꿔주는 함수
def stc2(selectivesearch_output):
  bboxes = []
  for ss in selectivesearch_output:
    bboxes.append([ss[0],ss[1],ss[0]+ss[2],ss[1]+ss[3]])
  return bboxes

#2 하나의 이미지 path를 받아 ROI 출력하는 함수

In [138]:
# 2. 하나의 이미지 path를 받아 모든 ROI의 768차원 임베딩을 생성하는 함수
# 3까지 같이해보기
## scale은 알고리즘이 찾는 ROI의 최소크기 
## min_size는 그 중 선택할 ROI의 최소크기 
## 
def ROI(image_filename, scale=2000, min_size=2000):
  img = Image.open(data_path+image_filename)
  img_np = np.array(img)
  _, regions = selective_search(img_np, scale=scale, min_size=min_size)
  bboxes = stc(regions)

  img_features = torch.empty(1,768).to(device)
  roi_image = []
  for bbox in bboxes:
    roi = img.crop((bbox[0],bbox[1],bbox[2],bbox[3]))
    roi_image.append(roi)
    img_input = (preprocess(roi).unsqueeze(0).to(device))
    with torch.no_grad():
     img_feature = model.encode_image(img_input)
     img_features = torch.cat((img_features, img_feature))
  img_features = img_features[1:,]
  return img_features, roi_image, bboxes

# 3. 모든 ROI 와 각 category 사이의 코사인 유사도 구하기
## text features는 이미 normalized됐다고 가정



In [48]:
# bbox, gt, cls를 리턴하는 함수
## 모든 ROI 중 제일 비슷한 cls와의 유사도가 threshold를 넘는 roi 빼고 suppress.
## 그렇게 구한 BBOX 별로 gt와 IOU를 계산해 일정 최대 IOU를 갖는 GT와 Match
## 이미지에 있는 gt를 각자의 cls와 함께 저장한다.
def scores(features):
  features /= features.norm(dim=-1, keepdim=True)
  text_features_norm = text_features / text_features.norm(dim=-1, keepdim=True)
  sim = torch.mm(features, text_features.T)
  # 각 클래스 텍스트 임베딩과의 cos sim
  probs = sim.softmax(dim=1)
  # 각 bbox별로 cls할당
  values, idx_names = torch.max(probs, dim=1)
  
  return idx_names, values

# NMS를 위해 bboxes, values, idx_names를 values를 기준으로 sort해주는 함수


In [49]:
def sort_for_nms(bboxes, values, idx_names):
  sort_idx = sorted(range(len(values)), key=lambda k: values[k], reverse=True)
  bboxes_output = torch.tensor(bboxes)[sort_idx]
  values_output = values[sort_idx]
  idx_names_output = idx_names[sort_idx]
  return bboxes_output, values_output, idx_names_output

# 4. NMS
1. NMS 
- Scores 함수에서 max(probs)(가장 비슷한 cls와 유사도)가 0.04 이상인 box만 남기기
- 이후 남은 박스들에 대해 NMS진행해 또 거르기

  --> 남은 박스의 인덱스를 활용해 ROI함수의 output인 bboxes에서 해당 박스들의 좌표를 가져와 keep






In [50]:
# IOU 구하는 함수 
def IoU(box1, box2):
    # box = (x1, y1, x2, y2)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter+1e-7)
    return iou

In [243]:
# NMS 해주는 함수
def NMS(bboxes, values, idx_names, conf_threshold=0.03, iou_threshold=0.5):
  values = values.cpu()
  # confidence가 낮은(기본값 0.04) bbox는 제거
  idx = np.where((values>=conf_threshold))[0]
  # NMS
  bbs_filtered=np.array(bboxes)[idx]
  values_filtered = values.detach().numpy()
  idx_names_filtered = idx_names.cpu().numpy()

  sup = torch.ones(len(bbs_filtered))
  
  for i in range(len(bbs_filtered)):
    for j in range(1, len(bbs_filtered)):
      iou = IoU(bbs_filtered[i],bbs_filtered[j])
      if (iou>=iou_threshold) and (idx_names_filtered[i]==idx_names_filtered[j]):
        sup[j] = 0
  sup_idx = np.where(sup)
  bbs_sup = bbs_filtered[sup_idx]
  values_sup = values_filtered[sup_idx]
  idx_names_sup = idx_names_filtered[sup_idx]
  return bbs_sup, values_sup, idx_names_sup



#5. Matching, 결과 기록

- 하나의 이미지에서 t개의 gt 불러오기
- 1번에서 구한 d개의 detection 불러오기

- 이미지에서 t개의 gt를 불러오기
  - d개의 detection과 t개의 gt를 count_det와 count_gt에 저장
- 1에서 구한 bb를 score 내림차순 정렬
- 각 bb마다 IOU가 가장 높은 gt를 match
  - iou가 threshold 이상
    - 여기서 아니면 correct = 0
  - 두 box의 cls가 같음
    - 여기서 아니면 correct = 0
  - 해당 gt에 이미 매칭된 bbox가 없음
    - 해당 bbox가 해당하는 correct = 1
    - 여기서 아니면 correct = 0
  

In [244]:
# 이미지 이름은 os.listdir(data_path)[i]로 줄 것
def matcher(bboxes_sup, values_sup, idx_names_sup, image_filename ):
  img = Image.open(data_path+image_filename)
  im_id = int(image_filename[:-4])
  # gt 가져오기
  gt_list = coco.getAnnIds(imgIds=im_id)
  gt_gtbb_nms=[]
  gtbbs=[]
  for gt in gt_list:
    gtbb = coco.loadAnns(gt)[0]['bbox']
    nms = coco.loadAnns(gt)[0]['category_id']
    gt_gtbb_nms.append([gt, stc2([gtbb])[0], nms])
    gtbbs.append(gtbb)


  iou=torch.empty(len(gt_gtbb_nms),len(bboxes_sup))
  for i,gt in enumerate(gt_gtbb_nms):
   for j, bb in enumerate(bboxes_sup):
     iou[i,j] = IoU(gt[1], bb)
  confs, indices = torch.max(iou, dim=0)
  names_bb, confs, indices = sort_for_nms(idx_names_sup, confs,indices)
  names_gt = torch.tensor([gt_gtbb_nms[idx][2] for idx in indices])                      
  correct = torch.eq(names_bb,names_gt)
  return sum(correct)
  


# Result
  - 모든 이미지에 대해 test한 결과mAP가 0에 수렴함. (=정답을 맞춘 경우가 거의 없음)
  - 처음 20개 결과를 출력해보면 정답의 개수가 매우 적다.
    - Image 하나당 정답을 맞춘 경우(IOU >=0.5)가 대부분 0이다.

In [247]:
result = []
for img in os.listdir(data_path)[0:20]:
  features, rois, bboxes = ROI(img, scale=2000, min_size=2000)
  idx_names, values = scores(features)
  bboxes_sorted, values_sorted, idx_names_sorted = sort_for_nms(bboxes, values, idx_names)
  bboxes_sup, values_sup, idx_names_sup = NMS(bboxes_sorted, values_sorted, idx_names_sorted, conf_threshold=0.03, iou_threshold=2)
  correct = matcher(bboxes_sup, values_sup, idx_names_sup, img)
  result.append([img, correct])



In [248]:
result

[['000000140076.jpg', tensor(0)],
 ['000000218439.jpg', tensor(0)],
 ['000000571598.jpg', tensor(0)],
 ['000000023899.jpg', tensor(0)],
 ['000000379842.jpg', tensor(0)],
 ['000000575081.jpg', tensor(0)],
 ['000000320642.jpg', tensor(0)],
 ['000000075612.jpg', tensor(0)],
 ['000000383842.jpg', tensor(0)],
 ['000000016958.jpg', tensor(1)],
 ['000000091500.jpg', tensor(0)],
 ['000000183049.jpg', tensor(0)],
 ['000000124659.jpg', tensor(0)],
 ['000000195842.jpg', tensor(0)],
 ['000000458663.jpg', tensor(0)],
 ['000000181753.jpg', tensor(0)],
 ['000000003934.jpg', tensor(0)],
 ['000000013923.jpg', tensor(1)],
 ['000000154358.jpg', tensor(0)],
 ['000000479248.jpg', tensor(0)]]